# Graphic user interfaces for data reduction.
``ess.reduce.parameter.Parameter`` is an interface between ``params`` of ``sciline.Pipeline``(workflow) and GUI components.

In this page, we will explain how to implement new Parameter and widgets and map those components with each other.

## Registries: How to map ``Parameter``, ``Workflow`` and ``Pipeline``.
Parameters, workflows and widgets should be registered once they are implemented
in order to be used automatically by workflow-widget building helper.

The helper uses registries to
- Find workflows
- Find input arguments that can be set by widget
- Find output types that can be computed by widget
- Map ``Parameter`` and ``Pipeline.params`` to build smaller widgets
- Set ``Pipeline.params`` based on the inputs of corresponding widgets

## Tutorial: Implement Random Distribution Histogram Workflow GUI
0. Prepare ``workflow`` interface that returns a ``sciline.Pipeline``
    We are going to use this random number histogram workflow as an example.
    We would like to implement a widget that users can put ``NumBins`` as an input.

    If you want to specify ``typical_outputs``, the ``Pipeline`` object should have a property called ``typical_outputs: tuple[type, ...]``.
    If it does not have a ``typical_outputs``, leaf nodes will be used as typical outputs.

In [ ]:
import scipp as sc
import sciline as sl
from typing import NewType
import numpy as np


NumBins = NewType('NumBins', int)
Histogram = NewType('Histogram', sc.Variable)


def histogram(num_bins: NumBins) -> Histogram:
    rng = np.random.default_rng()
    events = sc.array(dims=['event'], values=rng.normal(size=500))
    return Histogram(events.hist(event=num_bins))


def RandomDistributionWorkflow() -> sl.Pipeline:
    wf = sl.Pipeline(providers=(histogram,))
    # wf.typical_outputs = (Histogram, )  # Can be skipped since it's the only leaf node.
    return wf


RandomDistributionWorkflow()


1. Register workflow to the ``workflow_registry``.

In [ ]:
from ess.reduce.workflow import register_workflow

register_workflow(RandomDistributionWorkflow)

2. Register ``domain-type``-``Parameter`` instance mapping to ``parameter_registry``.

``Parameter`` and its subclasses have a class method ``from_type`` that helps to create a new parameter instance for a specific domain type.

There are various parameter types already exist in ``ess.reduce.parameter`` module, but here we will show you how to make a new one.

Once you know which ``Parameter`` to use for the specific ``domain-type``, 

you can register the new **parameter instance** to the **domain type** in the ``parameter_registry``.

In [ ]:
from ess.reduce.parameter import parameter_registry, Parameter


class BinNumberParameter(Parameter): ...


parameter_registry[NumBins] = BinNumberParameter(
    name=NumBins.__name__,
    description='Number of bins in the histogram',
    default=NumBins(10),
)

# You can also use ``from_type``(class method) helper to instantiate the parameter.
# parameter_registry[NumBins] = BinNumberParameter.from_type(NumBins)

3. Register ``type[Parameter]`` - ``type[Widget]`` mapping to ``create_parameter_widget`` distpatch.



In [ ]:
from ess.reduce.widgets import create_parameter_widget
import ipywidgets as widgets


@create_parameter_widget.register(BinNumberParameter)
def scalar_parameter_widget(param: BinNumberParameter):
    return widgets.IntText(
        value=param.default, description=param.name, tooltip=param.description
    )


### Example UI from the tutorial.

In [ ]:
from ess.reduce.ui import layout
# Select Workflow
_workflow_name, _workflow_obj = layout.children[0].children[0].options[0]
layout.children[0].children[0].value = _workflow_obj
del _workflow_name, _workflow_obj
# Set Output Parameter
layout.children[0].children[1].options[0]
layout.children[0].children[1].value = (layout.children[0].children[1].options[0],)
# Click ``Generate Parameters`` button
layout.children[0].children[3].children[0].click()
# Click ``Run Workflow`` button
layout.children[1].children[0].click()

In [ ]:
layout